# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 53 new papers today
          20 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/19 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.00010


extracting tarball to tmp_2211.00010...

 done.


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2211.00010/MWM31Sats_gasContent.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2211.00429/mnras_guide.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2211.00485...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.00613


extracting tarball to tmp_2211.00613...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.00010-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.00010) | **Satellites of MW/M31-like galaxies with TNG50: quenched fractions, gas  content, and star formation histories**  |
|| Christoph Engler, et al. -- incl., <mark>Annalisa Pillepich</mark> |
|*Appeared on*| *2022-11-02*|
|*Comments*| *Submitted to MNRAS -- comments welcome. Part of a set of papers based on TNG50 MW/M31-like galaxies*|
|**Abstract**| We analyse the quenched fractions, gas content, and star formation histories of ~1200 satellite galaxies with M* >= 5x10^6 Msun around 198 Milky Way- (MW) and Andromeda-like (M31) hosts in TNG50, the highest-resolution run of the IllustrisTNG simulations. Satellites exhibit larger quenched fractions for smaller masses, at smaller distances to their host galaxy, and in the more massive M31-like compared to MW-like hosts. As satellites cross their host's virial radius, their gas content drops significantly: most satellites within 300 kpc do not contain any detectable gas reservoirs at z=0, unless they are massive like the Magellanic Clouds and M32. Nevertheless, their stellar assembly exhibits a large degree of diversity. On average, the cumulative star formation histories of satellites are more extended for brighter, more massive satellites with a later infall, and for those in less massive hosts. Based on these relationships, we can even infer infall periods for observed MW and M31 dwarfs: e.g. 0-4 Gyr ago for the Magellanic Clouds and Leo I, 4-8 and 0-2 Gyr ago for M32 and IC 10, respectively. Ram pressure stripping (in combination with tidal stripping) deprives TNG50 satellites of their gas reservoirs and ultimately quenches their star formation activity, even though only a few per cent of the present-day satellites around the 198 TNG50 MW/M31-like hosts appear as jellyfish. The typical time since quenching for currently quenched TNG50 satellites is 6.9 (+2.5)(-3.3) Gyr ago. The TNG50 results are consistent with the quenched fractions and stellar assembly of observed MW and M31 satellites, however, satellites of the SAGA survey exhibit lower quenched fractions than TNG50 and other, observed analogues. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.00020-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.00020) | **The Circumgalactic Medium of Milky Way-like Galaxies in the TNG50  Simulation -- I: Halo Gas Properties and the Role of SMBH Feedback**  |
|| Rahul Ramesh, Dylan Nelson, <mark>Annalisa Pillepich</mark> |
|*Appeared on*| *2022-11-02*|
|*Comments*| *Submitted to MNRAS. Part of a set of papers based on TNG50 MW/M31-like galaxies. Additional visuals at www.tng-project.org/ramesh22!*|
|**Abstract**| We analyze the physical properties of gas in the circumgalactic medium (CGM) of 132 Milky Way (MW)-like galaxies at $z=0$ from the cosmological magneto-hydrodynamical simulation TNG50, part of the IllustrisTNG project. The properties and abundance of CGM gas across the sample are diverse, and the fractional budgets of different phases (cold, warm, and hot), as well as neutral HI mass and metal mass, vary considerably. Over our stellar mass range of $10.5 < \rm{M}_\star / \rm{M}_\odot < 10.9$, radial profiles of gas physical properties from $0.15 < \rm{R/R_{\rm 200c}} < 1.0$ reveal great CGM structural complexity, with significant variations both at fixed distance around individual galaxies, and across different galaxies. CGM gas is multi-phase: the distributions of density, temperature and entropy are all multimodal, while metallicity and thermal pressure distributions are unimodal; all are broad. We present predictions for magnetic fields in MW-like halos: a median field strength of $|B|\sim\,1\mu$G in the inner halo decreases rapidly at larger distance, while magnetic pressure dominates over thermal pressure only within $\sim0.2 \times \RVIR$. Virial temperature gas at $\sim 10^6\,$K coexists with a sub-dominant cool, $< 10^5\,$K component in approximate pressure equilibrium. Finally, the physical properties of the CGM are tightly connected to the galactic star formation rate, in turn dependent on feedback from supermassive black holes (SMBHs). In TNG50, we find that energy from SMBH-driven kinetic winds generates high-velocity outflows ($\gtrsim 2000$ km/s), heats gas to super-virial temperatures ($> 10^{6.5-7}$ K), and regulates the net balance of inflows versus outflows in otherwise quasi-static gaseous halos. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.00036-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.00036) | **The merger and assembly histories of Milky Way- and M31-like galaxies  with TNG50: disk survival through mergers**  |
|| <mark>Diego Sotillo-Ramos</mark>, et al. -- incl., <mark>Annalisa Pillepich</mark>, <mark>Lukas Eisert</mark>, <mark>Lars Hernquist</mark> |
|*Appeared on*| *2022-11-02*|
|*Comments*| *Published by MNRAS. Check also Chen et al., Engler et al. and Ramesh et al., on arxiv.org today, for further analyses of Milky Way- and Andromeda-like galaxies using the cosmological simulation TNG50. Main figures: 2, 7, 16*|
|**Abstract**| We analyze the merger and assembly histories of Milky Way (MW) and Andromeda (M31)-like galaxies to quantify how, and how often, disk galaxies of this mass can survive recent major mergers (stellar mass ratio $\ge$ 1:4). For this, we use the cosmological magneto-hydrodynamical simulation TNG50 and identify 198 analog galaxies, selected based on their $z=0$ stellar mass ($10^{10.5-11.2} {\rm M_{\odot}}$), disky stellar morphology and local environment. Firstly, major mergers are common: 85 per cent (168) of MW/M31-like galaxies in TNG50 have undergone at least one major merger across their lifetime. In fact, 31 galaxies (16 per cent) have undergone a recent major merger, i.e. in the last 5 Gyr. The gas available during the merger suffices to either induce starbursts at pericentric passages or to sustain prolonged star formation after coalescence: in roughly half of the cases, the pre-existing stellar disk is destroyed because of the merger but reforms thanks to star formation. Moreover, higher merger mass ratios are more likely to destroy the stellar disks. In comparison to those with more ancient massive mergers, MW/M31-like galaxies with recent major mergers have, on average, somewhat thicker stellar disks, more massive and somewhat shallower stellar haloes, larger stellar ex-situ mass fractions, but similarly massive kinematically-defined bulges. All this is qualitatively consistent with the different observed properties of the Galaxy and Andromeda and with the constraints on their most recent major mergers, 8-11 and ~2 Gyr ago, respectively. According to contemporary cosmological simulations, a recent quiet merger history is not a pre-requisite for obtaining a relatively-thin stellar disk at $z=0$. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.00087-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.00087) | **Where are the extremely metal-poor stars in the Milky Way and Andromeda?  Expectations from TNG50**  |
|| <mark>Li-Hsin Chen</mark>, <mark>Annalisa Pillepich</mark>, Simon C. O. Glover, Ralf S. Klessen |
|*Appeared on*| *2022-11-02*|
|*Comments*| *Part of a set of papers based on TNG50 MW/M31-like galaxies. 15 pages, 9 figures. Revised manuscript to be submitted to MNRAS after receiving positive feedback*|
|**Abstract**| We analyse the location of extremely metal-poor stars (EMPs, [Fe/H]$ < -3$) in 198 Milky Way (MW)/M31-like galaxies at $z=0$ in the TNG50 simulation. Each system is divided into four kinematically-defined morphological stellar components based on stellar circularity and galactocentric distance, namely bulge, cold disk, warm disk, and stellar halo, in addition to satellites (with stellar mass $\ge 5\times10^6\,M_\odot$). According to TNG50 and across all simulated systems, the stellar halo of the main galaxy and satellites present the highest frequency of EMPs (largest $M_{\mathrm{EMP, comp}}$-to-$M_{\mathrm{tot, comp}}$ stellar mass ratio), and thus the highest chances of finding them. Such frequency is larger in lower-mass than high-mass satellites. Moreover, TNG50 predicts that the stellar halo of the main galaxy always hosts and thus contributes the majority of the EMPs of the system. Namely, it has the highest mass ratio of EMPs in it to all the EMPs in the system (largest $M_{\mathrm{EMP, comp}}$-to-$M_\mathrm{EMP} (<300\mathrm{kpc})$). However, notably, we also find that 33 MW/M31-like galaxies in TNG50 have cold disks that contribute more than 10 per cent to the total EMP mass, each with $\gtrsim 10^{6.5-7}\, M_\odot$ of EMPs in cold circular orbits. These qualitative statements do not depend on the precise definition of EMP stars, i.e. on the adopted metallicity threshold. The results of this work provide a theoretical prediction for the location of EMP stars from both a spatial and kinematic perspective and across an unprecedented number of well-resolved MW/M31-like systems. |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.00429-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.00429) | **The survival of stellar discs in Fornax-like environments, from TNG50 to  real galaxies**  |
|| Pablo M. Galán-de Anta, et al. -- incl., <mark>A. Pillepich</mark>, <mark>F. Pinna</mark> |
|*Appeared on*| *2022-11-02*|
|*Comments*| *12 pages, 7 figures. Accepted on 21 of October, published on 25 of October*|
|**Abstract**| We study the evolution of kinematically-defined stellar discs in 10 Fornax-like clusters identified in the TNG50 run from the IllustrisTNG suite of cosmological simulations. We considered disc galaxies with present-day stellar mass $M_{\star}\geq 3 \times 10^{8} M_{\odot}$ and follow their evolution since first entering their host cluster. Very few stellar discs survive since falling in such dense environments, ranging from 40% surviving to all being disrupted. Such survival rates are consistent with what reported earlier for the two more massive, Virgo-like clusters in TNG50. In absolute terms, however, the low number of present-day disc galaxies in Fornax-like clusters could be at odds with the presence of three edge-on disc galaxies in the central regions of the actual Fornax cluster, as delineated by the Fornax3D survey. When looking at the Fornax analogues from random directions and with the same selection function of Fornax3D, the probability of finding three edge-on disc galaxies in any one Fornax-like cluster in TNG50 is rather low, albeit not impossible. We also compared the stellar-population properties near the equatorial plane derived from integral-field spectroscopy for the three edge-ons in Fornax to similar line-of-sight integrated values for present-day disc galaxies in TNG50. For one of these, the very old and metal-rich stellar population of its disc cannot be matched by any the disc galaxies in TNG50, including objects in the field. We discuss possible interpretations of these findings, while pointing to future studies on passive cluster spirals as a way to further test state-of-the-art cosmological simulations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.00485-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.00485) | **Relating galaxies across different redshift to study galaxy evolution**  |
|| Kai Wang, Houjun Mo, <mark>Cheng Li</mark>, Yangyao Chen |
|*Appeared on*| *2022-11-02*|
|*Comments*| *15 pages, 10+3 figures, submitted to MNRAS, comments are welcome*|
|**Abstract**| We propose a general framework leveraging the halo-galaxy connection to link galaxies observed at different redshift in a statistical way, and use the link to infer the redshift evolution of the galaxy population. Our tests based on hydrodynamic simulations show that our method can accurately recover the stellar mass assembly histories up to $z\sim 3$ for present star-forming and quiescent galaxies down to $10^{10}h^{-1}M_{\odot}$. Applying the method to observational data shows that the stellar mass evolution of the main progenitors of galaxies depends strongly on the properties of descendants, such as stellar mass, halo mass, and star formation states. Galaxies hosted by low-mass groups/halos at the present time have since $z\sim 1.8$ grown their stellar mass $\sim 2.5$ times as fast as those hosted by massive clusters. This dependence on host halo mass becomes much weaker for descendant galaxies with similar star formation states. Star-forming galaxies grow about 2-4 times faster than their quiescent counterparts since $z\sim 1.8$. Both TNG and EAGLE simulations over-predict the progenitor stellar mass at $z>1$, particularly for low-mass descendants. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.00613-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.00613) | **Building an Acceleration Ladder with Tidal Streams and Pulsar Timing**  |
|| Peter Craig, Sukanya Chakrabarti, <mark>Robyn E. Sanderson</mark>, Farnik Nikakhtar |
|*Appeared on*| *2022-11-02*|
|*Comments*| *8 pages, 4 figures, 1 table. Submitted to ApJ Letters*|
|**Abstract**| We analyze stellar streams in action-angle coordinates combined with recent local direct acceleration measurements to provide joint constraints on the potential of our Galaxy. Our stream analysis uses the Kullback-Leibler divergence with a likelihood analysis based on the two-point correlation function. We provide joint constraints from pulsar accelerations and stellar streams for local and global parameters that describe the potential of the Milky Way (MW). Our goal is to build an ``acceleration ladder", where direct acceleration measurements that are currently limited in dynamic range are combined with indirect techniques that can access a much larger volume of the MW. To constrain the MW potential with stellar streams, we consider the Palomar 5, Orphan, Nyx, Helmi and GD1 streams. Of the potential models that we have considered here, the preferred potential for the streams is a two-component Staeckel potential. We also compare the vertical accelerations from stellar streams and pulsar timing, defining a function $f(z) = \alpha_{1pulsar}z - \frac{\partial\Phi}{\partial z}$, where $\Phi$ is the MW potential determined from stellar streams, and $\alpha_{1~\rm pulsar}z$ is the vertical acceleration determined from pulsar timing observations. Our analysis indicates that the Oort limit determined from streams is consistently (regardless of the choice of potential) lower than that determined from pulsar timing observations. The calibration we have derived here may be used to correct the estimate of the acceleration from stellar streams. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.00152-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.00152) | **Multi-scale physical properties of NGC 6334 as revealed by relative  orientations between magnetic fields, density gradients, velocity gradients,  and gravity**  |
|| <mark>Junhao Liu</mark>, et al. -- incl., <mark>Qizhou Zhang</mark>, <mark>Hauyu Baobab Liu</mark>, <mark>Zhi-Yun Li</mark>, <mark>Shanghuo Li</mark> |
|*Appeared on*| *2022-11-02*|
|*Comments*| *27 pages, 19 figures. Submitted to ApJ. Revisions on the manuscript will be made during and after peer review*|
|**Abstract**| We present ALMA dust polarization and molecular line observations toward 4 clumps (I(N), I, IV, and V) in the massive star-forming region NGC 6334. In conjunction with large-scale dust polarization and molecular line data from JCMT, Planck, and NANTEN2, we make a synergistic analysis of relative orientations between magnetic fields ($\theta_{\mathrm{B}}$), column density gradients ($\theta_{\mathrm{NG}}$), local gravity ($\theta_{\mathrm{LG}}$), and velocity gradients ($\theta_{\mathrm{VG}}$) to investigate the multi-scale (from $\sim$30 pc to 0.003 pc) physical properties in NGC 6334. We find that the relative orientation between $\theta_{\mathrm{B}}$ and $\theta_{\mathrm{NG}}$ changes from statistically more perpendicular to parallel as column density ($N_{\mathrm{H_2}}$) increases, which is a signature of trans-to-sub-Alfv\'{e}nic turbulence at complex/cloud scales as revealed by previous numerical studies. Because $\theta_{\mathrm{NG}}$ and $\theta_{\mathrm{LG}}$ are preferentially aligned within the NGC 6334 cloud, we suggest that the more parallel alignment between $\theta_{\mathrm{B}}$ and $\theta_{\mathrm{NG}}$ at higher $N_{\mathrm{H_2}}$ is because the magnetic field line is dragged by gravity. At even higher $N_{\mathrm{H_2}}$, the angle between $\theta_{\mathrm{B}}$ and $\theta_{\mathrm{NG}}$ or $\theta_{\mathrm{LG}}$ transits back to having no preferred orientation or statistically slightly more perpendicular, suggesting that the magnetic field structure is impacted by star formation activities. A statistically more perpendicular alignment is found between $\theta_{\mathrm{B}}$ and $\theta_{\mathrm{VG}}$ throughout our studied $N_{\mathrm{H_2}}$ range, which indicates a trans-to-sub-Alfv\'{e}nic state at small scales as well. The normalised mass-to-flux ratio derived from the polarization-intensity gradient (KTH) method increases with $N_{\mathrm{H_2}}$. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.00236-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.00236) | **The Age and Erosion Rate of Young Sedimentary Rock on Mars**  |
|| <mark>An Y. Li</mark>, Edwin S. Kite, Katarina Keating |
|*Appeared on*| *2022-11-02*|
|*Comments*| *17 pages, 12 figures*|
|**Abstract**| The Medusae Fossae Formation (MFF) is an enigmatic sedimentary unit near the equator of Mars, with an uncertain formation process and absolute age. Due to the heavily wind-eroded surface, it is difficult to determine the absolute model age of the MFF using a one-parameter model based on the crater size-frequency distribution function with existing crater count data. We create a new two-parameter model that estimates both age and a constant erosion rate ($\beta$) by treating cratering as a random Poisson process. Our study uses new crater count data collected from Context Camera imagery for both the MFF and other young equatorial sedimentary rock. Based on our new model, the Central MFF formed $>$1.5 Gyr ago and had low erosion rates ($<$650 nm yr$^{-1}$), whereas the East MFF, Far East MFF, and Zephyria Planum most likely formed $<$1.5 Gyr ago and had higher erosion rates ($>$740 nm $^{-1}$). The top of Aeolis Mons (informally known as Mount Sharp) in Gale Crater and Eastern Candor have relatively young ages and low erosion rates. Based on the estimated erosion rates (since fast erosion permits metastable shallow ice), we also identify several sites, including Zephyria Planum, as plausible locations for shallow subsurface equatorial water ice that is detectable by gamma-ray spectroscopy or neutron spectroscopy. In addition to confirming $<$1.5 Gyr sedimentary rock formations on Mars, and distinguishing older and younger MFF sites, we find that fast-eroding locations have younger ages and MFF locations with slower erosion have older best-fit ages. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2211.00010.md
    + _build/html/tmp_2211.00010/./fquench_vs_mstar_main_SAGAII_incELVES.png
    + _build/html/tmp_2211.00010/./fquench_vs_mstar_incScatter.png
    + _build/html/tmp_2211.00010/./fquench_vs_mstar_compSAGAII_alt.png
    + _build/html/tmp_2211.00010/./stellarAssembly_vs_VMag_MWM31.png
    + _build/html/tmp_2211.00010/./SFH_perMstar_perInfall_MWlikeHosts.png
exported in  _build/html/2211.00020.md
    + _build/html/tmp_2211.00020/./figures/fig6.png
    + _build/html/tmp_2211.00020/./figures/fig6Cbar.png
    + _build/html/tmp_2211.00020/./figures/fig4_2.png
    + _build/html/tmp_2211.00020/./figures/fig4_2Cbar.png
    + _build/html/tmp_2211.00020/./figures/histPlots.png
    + _build/html/tmp_2211.00020/./figures/fig6Cbar.png
exported in  _build/html/2211.00036.md
    + _build/html/tmp_2211.00036/./Figures/TNG50-1_550934_73_MajorMerger1_starAndGasMass_sfrLog_dist_circ_starsCirc_images.png
    + _build/html/tmp_2211.00036/./Figures/circularitiesZ0vsTbirt

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\MSUN}{{\rm M}_{\sun}}$
$\newcommand{\MHOST}{M_{\rm 200c}}$
$\newcommand{\RHOST}{R_{\rm 200c}}$
$\newcommand{\MSTAR}{M_{\rm *}}$
$\newcommand{\ap}[1]{\textcolor{magenta}{#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\MSUN$}{{\rm M}_{\sun}}$
$\newcommand{$\MHOST$}{M_{\rm 200c}}$
$\newcommand{$\RHOST$}{R_{\rm 200c}}$
$\newcommand{$\MSTAR$}{M_{\rm *}}$
$\newcommand{$\ap$}[1]{\textcolor{magenta}{#1}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Satellites of MW/M31-like galaxies with TNG50: quenched fractions, gas content, and star formation histories

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.00010-b31b1b.svg)](https://arxiv.org/abs/2211.00010)<mark>Appeared on: 2022-11-02</mark> - _Submitted to MNRAS -- comments welcome. Part of a set of papers based on TNG50 MW/M31-like galaxies_

</div>
<div id="authors">

Christoph Engler, et al. -- incl., <mark><mark>Annalisa Pillepich</mark></mark>

</div>
<div id="abstract">

**Abstract:** We analyse the quenched fractions, gas content, and star formation histories of$\sim$1200 satellite galaxies with$M_* \geq 5 \times 10^6 $\MSUN$$around 198 Milky Way- (MW) and Andromeda-like (M31) hosts in TNG50, the highest-resolution run of the IllustrisTNG simulations. Satellites exhibit larger quenched fractions for smaller masses, at smaller distances to their host galaxy, and in the more massive M31-like compared to MW-like hosts. As satellites cross their host’s virial radius, their gas content drops significantly: most satellites within$300 {\rm kpc}$do not contain any detectable gas reservoirs at$z = 0$, unless they are massive like the Magellanic Clouds and M32. Nevertheless, their stellar assembly exhibits a large degree of diversity. On average, the cumulative star formation histories of satellites are more extended for brighter, more massive satellites with a later infall, and for those in less massive hosts. Based on these relationships, we can even infer infall periods for observed MW and M31 dwarfs: e.g.$0-4 {\rm Gyr}$ago for the Magellanic Clouds and Leo I,$4-8$and$0-2 {\rm Gyr}$ago for M32 and IC 10, respectively. Ram pressure stripping (in combination with tidal stripping) deprives TNG50 satellites of their gas reservoirs and ultimately quenches their star formation activity, even though only a few per cent of the present-day satellites around the 198 TNG50 MW/M31-like hosts appear as jellyfish. The typical time since quenching for currently quenched TNG50 satellites is$6.9\substack{+2.5\\-3.3}$Gyr ago. The TNG50 results are consistent with the quenched fractions and stellar assembly of observed MW and M31 satellites, however, satellites of the SAGA survey exhibit lower quenched fractions than TNG50 and other, observed analogues.

</div>

<div id="div_fig1">

<img src="tmp_2211.00010/./fquench_vs_mstar_main_SAGAII_incELVES.png" alt="Fig4.1" width="33%"/><img src="tmp_2211.00010/./fquench_vs_mstar_incScatter.png" alt="Fig4.2" width="33%"/><img src="tmp_2211.00010/./fquench_vs_mstar_compSAGAII_alt.png" alt="Fig4.3" width="33%"/>

**Figure 4. -** {\bf Quenched fractions of satellite galaxies with $\mathbf{M_{\boldsymbol{*}} \boldsymbol{\geq} 5 \boldsymbol{\times} 10^{6} {\bf M}_{\boldsymbol{\odot}0}}$ around MW/M31-like hosts in TNG50, the observed MW and M31, ELVES, and SAGA-II.}{\it Top panel:} median quenched fractions of TNG50 satellites stacked across all hosts (curves) and of observed satellites (points). In each comparison, we impose the respective observations' selection criteria on TNG50: galaxies within $300 {\rm kpc}$ (3D) of their host for the MW and M31 (black curve), galaxies within $300 {\rm kpc}$(2D) and $\pm 500 {\rm kpc}$ along the line of sight for ELVES (red curve), and galaxies within $300 {\rm kpc}$(2D) with line-of-sight velocities of $\pm 275 {\rm km s}^{-1}$ that have been matched to the absolute $r$-band magnitude $M_{\rm r}$ of each observed SAGA-II satellite for the SAGA survey (pink curve). The black points denote the quenched fractions of the observed MW and M31 satellites \protect\citep{Wetzel2015} while the red and pink points correspond to the stacked satellites of ELVES \citep{Carlsten2022} and SAGA-II \protect\citep{Mao2021}. While our fiducial quenched definition is based on the satellites' distance to the SFMS (solid curves), \protect\cite{Wetzel2015} employ a gas fraction criterion for LG satellites, \protect\cite{Carlsten2022} determine star formation activity based on their satellites' morphology, and the quenched fractions of \protect\cite{Mao2021} are based on H$\alpha$ equivalent widths. Thus, the solid curves represent a comparison at face value. We adopt the gas fraction-based criterion of \protect\cite{Wetzel2015}(dotted, black curve) to more directly compare to their observations. {\it Bottom left panel:} satellite quenched fractions including host-to-host variations. The thin, grey curves in the background denote the quenched fractions of satellites around individual TNG50 MW/M31-like hosts while the thick, black curve and the grey shaded area correspond to their mean and scatter as $16^{\rm th}$ and $84^{\rm th}$ percentiles. {\it Bottom right panel:} effects of host and satellite selections on the quenched fractions of the TNG50 SAGA-II analogue sample using our fiducial quenched definition. We show the quenched fractions for satellites around both TNG50 MW/M31-like and SAGA-like hosts according to the spatial SAGA-II selection criteria (dark and medium purple solid curves, respectively), and for the smaller sample of TNG50 satellites that are matched to the SAGA-II satellites based on their absolute $r$-band magnitude (pink solid curve, as in the top panel). The dashed light pink curve gives the quenched fractions predicted by TNG50 for SAGA-like satellites including an implicit selection cut on their surface brightness as suggested by \protect\citet[][see text for more details]{Font2022}. We illustrate the effects of different quenched definitions by adopting the gas fraction-based definition for our matched SAGA-like satellite sample (dotted vs. solid pink curves). (*fig:fquench*)

</div>
<div id="div_fig2">

<img src="tmp_2211.00010/./stellarAssembly_vs_VMag_MWM31.png" alt="Fig11" width="100%"/>

**Figure 11. -** {\bf Satellite stellar assembly times as a function of absolute $V$-band magnitude and stellar mass.} We define satellites (circles) as all galaxies with $M_* \geq 5 \times 10^6 $\MSUN$$ within $300 {\rm kpc}$ of either TNG50 MW-like hosts (left panels) or M31-like hosts (right panels) and define their stellar assembly in terms of $\tau_{90}$(top panels) and $\tau_{50}$(bottom panels), i.e. the lookback times at which the satellites had assembled 90 and 50 per cent of their present-day stellar mass. Filled blue circles denote galaxies that are star-forming or in the green valley at $z=0$, according to their distance to the SFMS; empty circles represent quenched satellites. Furthermore, we compare our TNG50 satellites to the observed dwarfs of the MW and M31 \protect\citep[red and brown diamonds, respectively,][]{Weisz2013, Weisz2014a, Navabi2021}. (*fig:stellarAssembly_vs_Vmag*)

</div>
<div id="div_fig3">

<img src="tmp_2211.00010/./SFH_perMstar_perInfall_MWlikeHosts.png" alt="Fig9" width="100%"/>

**Figure 9. -** {\bf Cumulative star formation histories (SFHs) of satellites within $\mathbf{300 {\rm \bf kpc}}$ of TNG50 MW-like hosts (i.e. with $\mathbf{M_{\boldsymbol{*}} \boldsymbol{=} 10^{10.5} \boldsymbol{-} 10^{10.9} {\rm \bf M}_{\boldsymbol{\odot}}}$)}. Each row depicts a different bin in satellite stellar mass (decreasing from top to bottom) whereas each column corresponds to a different bin of time since infall (accretion time $t_{\rm acc}$, with early to late infallers from left to right). The thin, grey curves show the cumulative SFHs of individual TNG50 satellites. For bins containing at least ten satellite galaxies, we compute the median SFH and its scatter as $16^{\rm th}$ and $84^{\rm th}$ percentiles (black curves and grey shaded areas, respectively). The grey diamonds denote the median stellar assembly times $\tau_{10}$, $\tau_{50}$, and $\tau_{90}$, i.e.  the times at which 10, 50, and 90 per cent of the present-day stellar mass has been assembled. Furthermore, we compare the normalised, cumulative SFHs of TNG50 satellites to those of observed satellites around the MW (coloured curves): the LMC and SMC \protect\citep{Weisz2013}, as well as Sagittarius, Fornax, and Leo I \protect\citep{Weisz2014a} using the best fit SFHs and their 68 per cent confidence intervals due to random and systematic uncertainties. (*fig:sfh_MWlike*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\ap}[1]{{\color{magenta} #1}}$
$\newcommand{\}{MSUN}$
$\newcommand{\}{RVIR}$
$\newcommand{\}{RTC}$
$\newcommand{\}{MTC}$
$\newcommand{\}{MSTC}$
$\newcommand{\}{RFC}$
$\newcommand{\}{MFC}$
$\newcommand{\}{ROUT}$
$\newcommand{\}{RHALF}$
$\newcommand{\}{MVIR}$
$\newcommand{\}{MHALO}$
$\newcommand{\}{MSH}$
$\newcommand{\}{MS}$
$\newcommand{\}{AS}$
$\newcommand{\}{ADM}$
$\newcommand{\}{etal}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\ap$}[1]{{\color{magenta} #1}}$
$\newcommand{\}{MSUN}$
$\newcommand{\}{RVIR}$
$\newcommand{\}{RTC}$
$\newcommand{\}{MTC}$
$\newcommand{\}{MSTC}$
$\newcommand{\}{RFC}$
$\newcommand{\}{MFC}$
$\newcommand{\}{ROUT}$
$\newcommand{\}{RHALF}$
$\newcommand{\}{MVIR}$
$\newcommand{\}{MHALO}$
$\newcommand{\}{MSH}$
$\newcommand{\}{MS}$
$\newcommand{\}{AS}$
$\newcommand{\}{ADM}$
$\newcommand{\}{etal}$</div>



<div id="title">

# The Circumgalactic Medium of Milky Way-like Galaxies in the TNG50 Simulation -- I: Halo Gas Properties and the Role of SMBH Feedback

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.00020-b31b1b.svg)](https://arxiv.org/abs/2211.00020)<mark>Appeared on: 2022-11-02</mark> - _Submitted to MNRAS. Part of a set of papers based on TNG50 MW/M31-like galaxies. Additional visuals at www.tng-project.org/ramesh22!_

</div>
<div id="authors">

Rahul Ramesh, Dylan Nelson, <mark><mark>Annalisa Pillepich</mark></mark>

</div>
<div id="abstract">

**Abstract:** We analyze the physical properties of gas in the circumgalactic medium (CGM) of 132 Milky Way (MW)-like galaxies at$z=0$from the cosmological magneto-hydrodynamical simulation TNG50, part of the IllustrisTNG project. The properties and abundance of CGM gas across the sample are diverse, and the fractional budgets of different phases (cold, warm, and hot), as well as neutral HI mass and metal mass, vary considerably. Over our stellar mass range of$10.5 < \rm{M}_\star / \rm{M}_\odot < 10.9$, radial profiles of gas physical properties from$0.15 < \rm{R/R_{\rm 200c}} < 1.0$reveal great CGM structural complexity, with significant variations both at fixed distance around individual galaxies, and across different galaxies. CGM gas is multi-phase: the distributions of density, temperature and entropy are all multimodal, while metallicity and thermal pressure distributions are unimodal; all are broad. We present predictions for magnetic fields in MW-like halos: a median field strength of$|B|\sim 1\mu$G in the inner halo decreases rapidly at larger distance, while magnetic pressure dominates over thermal pressure only within$\sim0.2 \times \RVIR$. Virial temperature gas at$\sim 10^6 $K coexists with a sub-dominant cool,$< 10^5 $K component in approximate pressure equilibrium. Finally, the physical properties of the CGM are tightly connected to the galactic star formation rate, in turn dependent on feedback from supermassive black holes (SMBHs). In TNG50, we find that energy from SMBH-driven kinetic winds generates high-velocity outflows ($\gtrsim 2000$km/s), heats gas to super-virial temperatures ($> 10^{6.5-7}$K), and regulates the net balance of inflows versus outflows in otherwise quasi-static gaseous halos.

</div>

<div id="div_fig1">

<img src="tmp_2211.00020/./figures/fig6.png" alt="Fig6.1" width="50%"/><img src="tmp_2211.00020/./figures/fig6Cbar.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Spherically-averaged radial profiles of six physical gas quantities as a function of (3D) galactocentric distance around TNG50 MW-like galaxies. We consider density (upper left), HI density (upper right), temperature (center left), metallicity (center right), thermal pressure (lower left) and entropy (lower right). Each panel shows the median profile across the galaxy sample (black line), while the thinner curves correspond to profiles of individual halos, colored by galaxy stellar mass. Such profiles represent the azimuthally-averages of the depicted integrated or mass-weighted gas property at the given radius. Whereas the temperature and entropy of gas is generally lower in the inner halo as compared to the outskirts, due to the presence of a second cooler component becoming dominant, no such trend is clearly seen in the other four panels. The galaxy-to-galaxy scatter of individual curves about the median profile is large, and often correlates well with stellar mass. (*fig:radial_profiles*)

</div>
<div id="div_fig2">

<img src="tmp_2211.00020/./figures/fig4_2.png" alt="Fig4.1" width="50%"/><img src="tmp_2211.00020/./figures/fig4_2Cbar.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** Selected integral properties of the CGM plotted as a function of galaxy stellar mass for all 132 MW-like galaxies from the TNG50 simulation, at $z=0$: total CGM gas mass (top-left), CGM cold gas (T $<10^{4.5}$K) mass (along with the median line of CGM HI mass; top-right panel), CGM metal mass (bottom-left), and CGM X-ray luminosity (0.5-2.0 keV; bottom right panel). The first three quantities depend only weakly on galaxy stellar mass, while X-ray luminosity rises sharply with increasing stellar mass of the galaxy. In each panel, the circular markers are color coded by the sSFR of the galaxy -- in all cases, we note that galaxies with a lower sSFR have smaller values of the corresponding quantity. In the top-left panel, we consider two different definitions for the CGM: variable volume (extends between 0.15 and 1.0 R$_{\rm{200,c}}$) and constant volume (extends between 30 and 200 kpc) -- the two definitions yield statistically similar results at the low mass end, while differences are apparent towards the high mass end. In all other panels, we adopt the variable volume definition, which is our fiducial choice. (*fig:cgm_props_vs_mass*)

</div>
<div id="div_fig3">

<img src="tmp_2211.00020/./figures/histPlots.png" alt="Fig8.1" width="50%"/><img src="tmp_2211.00020/./figures/fig6Cbar.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** Distributions of the physical properties of gas in the CGM of TNG50 MW-like galaxies at $z=0$(in each case, histograms employ 30 bins, and extend between the visible tick marks. Each (thin) curve corresponds to a single galaxy/halo, color showing its stellar mass, and accounts, as per definition, only of the gas between $0.15$ and $1.00 \times \rm{R_{vir}}$. The thicker curves show medians across halos binned into four galaxy stellar mass quartiles. While the metallicity and thermal pressure distributions are unimodal, gas density, HI density, temperature, and entropy are multimodal, suggesting that the CGM hosts multi-phase gas that forms distinct populations in density, temperature, and entropy. Only temperature and entropy show peaks with strong monotonic trends as a function of stellar mass, with more more massive galaxies hosting gas at higher temperatures and higher entropy in their CGM. (*fig:histPlots*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\DSR}[1]{\textcolor{red}{#1}}$
$\newcommand{\DSRc}[1]{\textit{\textcolor{red}{#1}}}$
$\newcommand{\DTA}[1]{\textcolor{cyan}{#1}}$
$\newcommand{\ap}[1]{\textcolor{magenta}{#1}}$
$\newcommand{\apn}[1]{\textcolor{red}{#1}}$
$\newcommand{\MS}{\rm{M}_{\odot}}$
$\newcommand{\MH}{\rm{M}_{\rm 200c}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\}{blankpage}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\DSR$}[1]{\textcolor{red}{#1}}$
$\newcommand{$\DSR$c}[1]{\textit{\textcolor{red}{#1}}}$
$\newcommand{$\DTA$}[1]{\textcolor{cyan}{#1}}$
$\newcommand{$\ap$}[1]{\textcolor{magenta}{#1}}$
$\newcommand{$\ap$n}[1]{\textcolor{red}{#1}}$
$\newcommand{$\MS$}{\rm{M}_{\odot}}$
$\newcommand{$\MH$}{\rm{M}_{\rm 200c}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\}{blankpage}$</div>



<div id="title">

# The merger and assembly histories of Milky Way- and M31-like galaxies with TNG50: disk survival through mergers

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.00036-b31b1b.svg)](https://arxiv.org/abs/2211.00036)<mark>Appeared on: 2022-11-02</mark> - _Published by MNRAS. Check also Chen et al., Engler et al. and Ramesh et al., on arxiv.org today, for further analyses of Milky Way- and Andromeda-like galaxies using the cosmological simulation TNG50. Main figures: 2, 7, 16_

</div>
<div id="authors">

<mark><mark>Diego Sotillo-Ramos</mark></mark>, et al. -- incl., <mark><mark>Annalisa Pillepich</mark></mark>, <mark><mark>Lukas Eisert</mark></mark>, <mark><mark>Lars Hernquist</mark></mark>

</div>
<div id="abstract">

**Abstract:** We analyze the merger and assembly histories of Milky Way (MW) and Andromeda (M31)-like galaxies to quantify how, and how often, disk galaxies of this mass can survive recent major mergers (stellar mass ratio$\ge$1:4).For this, we use the cosmological magneto-hydrodynamical simulation TNG50 and identify 198 analog galaxies, selected based on their$z=0$stellar mass ($10^{10.5-11.2} $\MS$$), disky stellar morphology and local environment.Firstly, major mergers are common: 85 per cent (168) of MW/M31-like galaxies in TNG50 have undergone at least one major merger across their lifetime.In fact, 31 galaxies (16 per cent) have undergone a recent major merger, i.e. in the last 5 Gyr. The gas available during the merger suffices to either induce starbursts at pericentric passages or to sustain prolonged star formation after coalescence: in roughly half of the cases, the pre-existing stellar disk is destroyed because of the merger but reforms thanks to star formation. Moreover, higher merger mass ratios are more likely to destroy the stellar disks.In comparison to those with more ancient massive mergers, MW/M31-like galaxies with recent major mergers have, on average, somewhat thicker stellar disks, more massive and somewhat shallower stellar haloes, larger stellar ex-situ mass fractions, but similarly massive kinematically-defined bulges. All this is qualitatively consistent with the different observed properties of the Galaxy and Andromeda and with the constraints on their most recent major mergers,$8-11$and  2 Gyr ago, respectively.According to contemporary cosmological simulations, a recent quiet merger history is not a pre-requisite for obtaining a relatively-thin stellar disk at$z=0$.

</div>

<div id="div_fig1">

<img src="tmp_2211.00036/./Figures/TNG50-1_550934_73_MajorMerger1_starAndGasMass_sfrLog_dist_circ_starsCirc_images.png" alt="Fig13" width="100%"/>

**Figure 13. -** Evolutionary tracks of an example TNG50 MW/M31-like galaxy whose last major merger destroys the stellar disk, but a new one reforms.
    In all panels, but the images at the bottom, values are shown as a function of cosmic time. The vertical thick solid line represents the time of the last major merger; the thin vertical line in the top panel denotes the time when the stellar mass ratio is evaluated.
    \textit{Top panel:} Time evolution of the gravitationaly-bound stellar and gas mass (solid and dashed line, respectively) for the primary and secondary progenitors.
    \textit{Second to fourth panels:} SFR of the main progenitor (fuchsia), distance between progenitors (red), and diskyness i.e. D/T mass ratio (blue -- see definition in Section \ref{sec:galStarProperties}).
    \textit{Fifth to seventh panels:} Distributions of the orbit circularities of stellar particles. From top to bottom: circularities of the in-situ stars along the main-progenitor branch of the galaxy; in-situ circularities evaluated at $z=0$ and shown as a function of the birth time of the stars; in-situ circularity evaluated at the time of birth and shown as as a function of birth time.
    \textit{Bottom panels:} Stellar density images of the main progenitors, centered at the main galaxy, face-on and an edge-on views, across snapshots separated by about 150 Myrs.
     (*fig:circularitiesLMMdestroysDisk*)

</div>
<div id="div_fig2">

<img src="tmp_2211.00036/./Figures/circularitiesZ0vsTbirth_BeforeDuringAfterLMMlast5Gyr_stacked.png" alt="Fig16.1" width="33%"/><img src="tmp_2211.00036/./Figures/scatter_insituLast5GyrVSTotal_z0.png" alt="Fig16.2" width="33%"/><img src="tmp_2211.00036/./Figures/scatter_exsituFromLMMVSinsituLast5Gyr_z0.png" alt="Fig16.3" width="33%"/>

**Figure 16. -** 
    Connection between star formation, orbital circularities of the stars and the amount of in-situ star formation triggered by the merger and of the ex-situ stars brought in by the mergers.
    \textit{Top}: circularities of the stellar orbits at the time of formation vs. at $z=0$, stacked for all 31 TNG50 MW/M31-like galaxies with a recent major merger, for stars formed before (before $t_{\rm{max}}$),  during (between $t_{\rm{max}}$ and $t_{\rm{merger}}$), and after the last major merger, from left to right. In all panels, stars in perfectly circular orbits have a circularity $=1$(or $-1$ if they are counter-rotating); stars in radial orbits have a circularity of $\sim0$. Histograms show the total stellar mass per galaxy born in the respective periods of time.
    \textit{Bottom left}: Fraction of in-situ stellar mass formed in the last 5 Gyr as a function of total stellar mass at $z=0$.
    \textit{Bottom right}: Ex-situ stellar mass brought during the last major merger vs. in situ stellar mass in the last 5 Gyr. In both cases, TNG50 MW/M31-like galaxies with a recent major merger (blue) are contrasted with those with at least one major merger across their history but irrespective of when (light blue). Most of the stars were born in circular orbits, this is generally the case both before and after the times of the last major merger and, to a lesser extent, during the last major merger; however, the orbits of the stars born after the last recent major merger remain mostly unaltered, i.e. in circular orbits, all the way to $z=0$. Furthermore, recent major mergers also seem to trigger more in-situ star formation in the resulting galaxy than in galaxies with more ancient mergers, in addition to bringing large amounts of ex-situ stellar mass. (*fig:circularitiesPrePostLMM*)

</div>
<div id="div_fig3">

<img src="tmp_2211.00036/./Figures/histogram_LMM_lookbackTime.png" alt="Fig12.1" width="25%"/><img src="tmp_2211.00036/./Figures/histogram_LMM_mergerDuration.png" alt="Fig12.2" width="25%"/><img src="tmp_2211.00036/./Figures/histogram_LMM_tmaxMassType4Ratio.png" alt="Fig12.3" width="25%"/><img src="tmp_2211.00036/./Figures/scatter_LMM_tmaxMassType4Ratio_vs_MassType4z0.png" alt="Fig12.4" width="25%"/>

**Figure 12. -** Characteristics of the last major mergers of MW/M31-like galaxies in TNG50 at $z=0$: major mergers. In all panels, MW/M31-like galaxies that experienced their final major merger in the last 5 Gyr are depicted in blue; MW/M31 analogues with their last major merger at any cosmic time are indicated in light blue.
    \textit{Top left}: Time of the last major merger (in lookback Gyr) for each of the 198 galaxies, in bins of $1 $Gyr: 30 MW/M31-like galaxies did not undergo any major mergers (orange bin); 95 galaxies (approximately 50 per cent) underwent their last major merger more than 9 Gyr ago ($z\gtrsim1.3)$; 31 galaxies have had a late major merger, i.e. as recent as in the last 5 Gyr. The occurrence of the latter is distributed in an approximately uniform way in the considered period of time.
    \textit{Top right}: Duration of the major mergers, defined as the time elapsed since the secondary reached its maximum in stellar mass ($t_\rmn{max}$) and the moment of coalescence -- bins span 330 million years. 77 per cent of the major mergers lasting longer than $1.5 $Gyr are recent.
    \textit{Bottom left}: Stellar-mass ratios between the secondary and the primary, at $t_\rmn{max}$. \textit{Bottom right}: Stellar mass of both progenitors, at $t_\rmn{max}$, versus the stellar mass of the galaxy at $z=0$. For the Galaxy and Andromeda, we show observational estimates with magenta and orange annotations, respectively: the mass-ratio and the time of the known last major merger are taken for the case of GES \citep[][]{Helmi2018, Gallart2019, Naidu2021} and for M32 \citep[][]{DSouza2018}.
     (*fig:lastMajorMerger*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\angstrom}{\textup{\AA}}$
$\newcommand{\lhnote}[1]{\textcolor{cyan}{{#1}}}$
$\newcommand{\msun}{{\rm M}_\odot}$
$\newcommand{\zsun}{Z_\odot}$
$\newcommand{\mste}{M_{\rm *}}$
$\newcommand{\feh}{\mathrm{[Fe/H]}}$
$\newcommand{\epz}{\epsilon_\mathrm{z}}$
$\newcommand{\empempall}{M_{\mathrm{EMP, comp}}-to-M_\mathrm{EMP} (<300\mathrm{kpc}) }$
$\newcommand{\empall}{M_{\mathrm{EMP, comp}}-to-M_{\mathrm{tot, comp}} }$
$\newcommand{\empallr}{M_{\mathrm{EMP}}(r)-to-M_{\mathrm{tot}}(r) }$
$\newcommand{\ap}[1]{\textcolor{magenta}{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\angstrom$}{\textup{\AA}}$
$\newcommand{$\lhnote$}[1]{\textcolor{cyan}{{#1}}}$
$\newcommand{$\msun$}{{\rm M}_\odot}$
$\newcommand{$\zsun$}{Z_\odot}$
$\newcommand{$\mste$}{M_{\rm *}}$
$\newcommand{$\feh$}{\mathrm{[Fe/H]}}$
$\newcommand{$\epz$}{\epsilon_\mathrm{z}}$
$\newcommand{$\empempall$}{M_{\mathrm{EMP, comp}}-to-M_\mathrm{EMP} (<300\mathrm{kpc}) }$
$\newcommand{$\empall$}{M_{\mathrm{EMP, comp}}-to-M_{\mathrm{tot, comp}} }$
$\newcommand{$\empall$r}{M_{\mathrm{EMP}}(r)-to-M_{\mathrm{tot}}(r) }$
$\newcommand{$\ap$}[1]{\textcolor{magenta}{#1}}$</div>



<div id="title">

# Where are the extremely metal-poor stars in the Milky Way and Andromeda? Expectations from TNG50

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.00087-b31b1b.svg)](https://arxiv.org/abs/2211.00087)<mark>Appeared on: 2022-11-02</mark> - _Part of a set of papers based on TNG50 MW/M31-like galaxies. 15 pages, 9 figures. Revised manuscript to be submitted to MNRAS after receiving positive feedback_

</div>
<div id="authors">

<mark><mark>Li-Hsin Chen</mark></mark>, <mark><mark>Annalisa Pillepich</mark></mark>, Simon C. O. Glover, Ralf S. Klessen

</div>
<div id="abstract">

**Abstract:** We analyse the location of extremely metal-poor stars (EMPs,$\feh < -3$) in 198 Milky Way (MW)/M31-like galaxies at$z=0$in the TNG50 simulation. Each system is divided into four kinematically-defined morphological stellar components based on stellar circularity and galactocentric distance, namely bulge, cold disk, warm disk, and stellar halo, in addition to satellites (with stellar mass$\ge5\times10^6 $\msun$$). According to TNG50 and across all simulated systems, the stellar halo of the main galaxy and satellites present the highest frequency of EMPs (largest$\empall$stellar mass ratio), and thus the highest chances of finding them. Such frequency is larger in lower-mass than high-mass satellites. Moreover, TNG50 predicts that the stellar halo of the main galaxy always hosts and thus contributes the majority of the EMPs of the system. Namely, it has the highest mass ratio  of EMPs in it to all the EMPs in the system (largest$\empempall$). However, notably, we also find that 33 MW/M31-like galaxies in TNG50 have cold disks that contribute more than 10 per cent to the total EMP mass, each with$\gtrsim 10^{6.5-7}  $\msun$$of EMPs in cold circular orbits. These qualitative statements do not depend on the precise definition of EMP stars, i.e. on  the adopted metallicity threshold. The results of this work provide a theoretical prediction for the location of EMP stars from both a spatial and kinematic perspective and across an unprecedented number of well-resolved MW/M31-like systems.

</div>

<div id="div_fig1">

<img src="tmp_2211.00087/./figs/Mstar_McompLowZ.png" alt="Fig8.1" width="50%"/><img src="tmp_2211.00087/./figs/fraction.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** Mass fraction of EMP stars in 198 TNG50 MW/M31 analogues in different morphological components\textbf{Stellar mass and mass fractions of EMPs in 198 TNG50 MW/M31 analogues across their different morphological components.} In the top panel, we give the amounts of stellar mass in EMP stellar populations predicted by TNG50 across and within the different galaxies. In the bottom left panel, we show the $\empall$ fraction, i.e. the frequency of EMPs on a component-by-component basis. In the bottom right, we show the $\empempall$ fractions, i.e. the contribution of each morphological component to the total mass in EMPs, i.e. across all components. Satellites belonging to each galaxy are considered as one component in the system (the "satellites", see Sec. \ref{sec:morph} for details). Bulge, cold disk, warm disk, stellar halo, and satellites in each MW/M31-like system are shown as red, blue, green, orange, and grey circles, respectively. The amount of EMPs across all the morphological components ($M_\mathrm{EMP} (<300\mathrm{kpc})$) is shown as black crosses.
    There are 6 MW/M31 analogues without any satellites and 1 MW/M31 analogue that has only one metal-enriched satellite. They are manually added in the bottom of the top panel in filled grey circles.
     (*fig:empfraction*)

</div>
<div id="div_fig2">

<img src="tmp_2211.00087/./figs/morph.png" alt="Fig3" width="100%"/>

**Figure 3. -** Kinematically-defined morphological decomposition of the simulated MW/M31-like galaxies adopted throughout.{\bf Kinematically-defined morphological decomposition of the simulated MW/M31-like galaxies adopted throughout.}{\it Top}: Cartoon plot of the morphological decomposition of the main galaxy's body, based on circularity and galactocentric distance of the stars: $r_\mathrm{cut} = 3.5$ kpc, $r_\mathrm{disk} = 6 \times r_\mathrm{disk scale length}$, and $r_\mathrm{halo} = 300$ kpc. In addition to the depicted components, namely bulge, cold disk, warm disk, and stellar halo, satellite galaxies are identified as gravitationally-bound substructures of stars within 300 kpc from the centre, via the $\textsc${Subfind} algorithm. In this paper, we only consider satellites more massive than $5\times10^6 $\msun$$ in stars.
    {\it Bottom}: stellar mass in the distinct morphological components of all 198 MW/M31-like systems of TNG50 at $z=0$. (*fig:morph_decomp*)

</div>
<div id="div_fig3">

<img src="tmp_2211.00087/./figs/MDF_All.png" alt="Fig4.1" width="33%"/><img src="tmp_2211.00087/./figs/MDF_Y20B21.png" alt="Fig4.2" width="33%"/><img src="tmp_2211.00087/./figs/MDF_Hayden15.png" alt="Fig4.3" width="33%"/>

**Figure 4. -** MDFs of TNG50 MW/M31 analogues{\bf Stellar metallicity distribution functions (MDFs) of MW/M31-like galaxies in TNG50.}{\it Top panels:} we show the MDFs of all 198 MW/M31 systems, across {\it all} their morphological components: disks, bulge, stellar halo and satellites, colour coded by galaxy stellar mass. The redder the colour, the higher the galaxy stellar mass. On the left, we show the cumulative fraction and on the right, we show the stellar mass. {\it Five bottom panels:} MDFs of the subsample TNG50 galaxies with mass more similar to the Milky Way (gray curves) overlaid to results from observations and thus with stars selected by height and radial distance to attempt to account for the surveys' selection functions: we report here the Milky Way's MDF by \citet{Youakim:2020aa}(green, top left), \citet{Bonifacio:2021aa}(magenta, top right), and by \citet{Hayden:2015aa}(shades of blue, three bottom panels, for three different height selections). For the middle left panel, we select stars that have heliocentric distances between 6 and 20 kpc, and at Galactic latitudes between 30 and 78 degrees. The solar position in each MW-like galaxy is randomly sampled from the galactic centre and we compute the mean MDF over 100 runs. The probability is then normalised to the total number of stars between $-4. < $\feh$ < -1.05$ to compare with \citet{Youakim:2020aa}. In the middle right panel, we select stars that are < 6 kpc from the galactic plane and < 14 kpc from the galactic centre. The probability is again normalised to the total number of stars between $-4. < $\feh$ < -1.05$ to compare with \citet{Bonifacio:2021aa}. In the bottom three panels, we group stars at different distances from the galactic plane to compare with \citet{Hayden:2015aa}, where the probabilities are normalised to total number of stars in each subset. We do not impose any additional selection function to TNG50 star particles but for the aforementioned geometrical spatial cuts that account for the effective footprints of the APOGEE survey. Note that the MDF in \citet{Hayden:2015aa} is not suitable for comparison below $\feh \leq -1$. (*fig:mdf*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

78  publications in the last 7 days.
	 _build/html/2211.00087.md
	 _build/html/2211.00036.md
	 _build/html/2211.00020.md
	 _build/html/2211.00010.md
	 _build/html/2210.17434.md
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers